## Data Engineering Capstone 1: Movies & Shows dataset

In this Jupyter Notebook example, we will demonstrate how to work with a dataset of Netflix TV Shows and Movies. We will perform normalization procedures on the data and load it into a database.

### Objectives

*   Practice setting up an RDBMS.
*   Practice ingesting data from CSV files.
*   Practice normalizing tables.

## Start
Let's import the libraries that we will use for our project:

In [1]:
import os
import pandas as pd  # Data manipulation and analysis library
from sqlalchemy import (  # SQL toolkit and Object-Relational Mapping (ORM)
    create_engine, text, exc, ForeignKey, ForeignKeyConstraint,
    Column, String, Integer, Float, CHAR, Text, Date
)
from sqlalchemy.orm import declarative_base, relationship, sessionmaker  # ORM-related tools
from sqlalchemy_utils import database_exists, create_database  # Additional SQLAlchemy utilities
import config as cfg  # Configuration file
import logging

pd.options.mode.copy_on_write = True

Let's describe our environment variables:

In [2]:
DATABASE = "movies_and_shows"
HOSTNAME = cfg.HOSTNAME
USER = cfg.USER
PASSWORD = cfg.PASSWORD
CONNECTION_STRING = f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DATABASE}"

And set up logging:

In [3]:
logging.basicConfig(filename='data_error.log', level=logging.INFO)

Let's create a new Engine instance. This Engine instance will serve as the interface to your MySQL database, allowing you to execute SQL commands and interact with the database within your Jupyter notebook:

In [4]:
engine = create_engine(CONNECTION_STRING)

Let's do a backend-specific testing to quickly determine if a database exists on the server. If not, create a database:

In [5]:
if not database_exists(engine.url):
    create_database(engine.url)
print("Database exists: ", database_exists(engine.url))

Database exists:  True


## Data Loading and Cleaning

Start by loading the Netflix TV Shows and Movies dataset from the provided CSV file into a Pandas DataFrame and peek into the dataset by displaying the first few rows using the .head() method to understand its structure and contents:

In [6]:
titles = pd.read_csv('../../datasets/raw_titles.csv')
titles

,index,id,title,type,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes
0,0,ts300399,Five Came Back: The Reference Films,SHOW,1945,TV-MA,48,['documentation'],['US'],1.0,NaN,NaN,NaN
1,1,tm84618,Taxi Driver,MOVIE,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0
2,2,tm127384,Monty Python and the Holy Grail,MOVIE,1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0
3,3,tm70993,Life of Brian,MOVIE,1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0
4,4,tm190788,The Exorcist,MOVIE,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5801,5801,tm1014599,Fine Wine,MOVIE,2021,NaN,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.9,39.0
5802,5802,tm1108171,Edis Starlight,MOVIE,2021,NaN,74,"['music', 'documentation']",[],NaN,NaN,NaN,NaN
5803,5803,tm1045018,Clash,MOVIE,2021,NaN,88,"['family', 'drama']","['NG', 'CA']",NaN,tt14620732,6.5,32.0
5804,5804,tm1098060,Shadow Parties,MOVIE,2021,NaN,116,"['action', 'thriller']",[],NaN,tt10168094,6.2,9.0


In [7]:
shows = titles[titles["type"] == "SHOW"]
shows = shows.reset_index(drop=True)
shows.index = shows.index + 1
shows.rename(columns = {'id':'show_id'}, inplace = True)
shows.drop(columns=['index', 'type', 'genres', 'production_countries', 'imdb_id', 'imdb_score', 'imdb_votes'], inplace=True)
shows

,show_id,title,release_year,age_certification,runtime,seasons
1,ts300399,Five Came Back: The Reference Films,1945,TV-MA,48,1.0
2,ts22164,Monty Python's Flying Circus,1969,TV-14,30,4.0
3,ts45948,Monty Python's Fliegender Zirkus,1972,TV-MA,43,1.0
4,ts20681,Seinfeld,1989,TV-PG,24,9.0
5,ts22082,Knight Rider,1982,TV-PG,51,4.0
...,...,...,...,...,...,...
2043,ts296698,Glimpses of a Future,2021,TV-PG,4,1.0
2044,ts297061,Masameer County,2021,TV-MA,23,1.0
2045,ts286386,The Big Day,2021,TV-MA,45,2.0
2046,ts307884,HQ Barbers,2021,TV-14,24,1.0


In [8]:
movies = titles[titles["type"] == "MOVIE"]
movies = movies.reset_index(drop=True)
movies.index = movies.index + 1
movies.rename(columns = {'id':'movie_id'}, inplace = True)
movies.drop(columns=['index', 'type', 'genres', 'production_countries', 'seasons', 'imdb_id', 'imdb_score', 'imdb_votes'], inplace=True)
movies

,movie_id,title,release_year,age_certification,runtime
1,tm84618,Taxi Driver,1976,R,113
2,tm127384,Monty Python and the Holy Grail,1975,PG,91
3,tm70993,Life of Brian,1979,R,94
4,tm190788,The Exorcist,1973,R,133
5,tm14873,Dirty Harry,1971,R,102
...,...,...,...,...,...
3755,tm1040816,Momshies! Your Soul is Mine,2021,NaN,108
3756,tm1014599,Fine Wine,2021,NaN,100
3757,tm1108171,Edis Starlight,2021,NaN,74
3758,tm1045018,Clash,2021,NaN,88


In [9]:
genres_exploded = titles[['id', 'genres']]

genres_exploded.loc[:, 'genres'] = genres_exploded['genres'].str.replace(r'\[|\]|\'', '', regex=True)
genres_exploded.loc[:, 'genres'] = genres_exploded['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

genres_exploded = genres_exploded.explode('genres')

genres = genres_exploded[['genres']].drop_duplicates().reset_index(drop=True)
genres.index = genres.index + 1
genres['genre_id'] = genres.index

genres_bridge = pd.merge(genres_exploded, genres, on='genres', how='left')
genres_bridge.rename(columns = {'index':'genre_id', 'id':'title_id'}, inplace = True)
genres_bridge.rename_axis('index', inplace=True)
genres_bridge.index = genres_bridge.index + 1
genres_bridge.drop(columns=['genres'], inplace=True)

genres_bridge.loc[genres_bridge['title_id'].str.startswith('tm'), 'movie_id'] = genres_bridge['title_id']
genres_bridge.loc[genres_bridge['title_id'].str.startswith('ts'), 'show_id'] = genres_bridge['title_id']
genres_bridge.drop(columns=['title_id'], inplace=True)

genres.drop(columns=['genre_id'], inplace=True)
genres.rename_axis('genre_id', inplace=True)
genres.rename(columns = {'genres':'genre'}, inplace = True)

del genres_exploded
genres

,genre
genre_id,
1,documentation
2,crime
3,drama
4,comedy
5,fantasy
6,horror
7,european
8,thriller
9,action


In [10]:
genres_bridge

,genre_id,movie_id,show_id
index,,,
1,1,NaN,ts300399
2,2,tm84618,NaN
3,3,tm84618,NaN
4,4,tm127384,NaN
5,5,tm127384,NaN
...,...,...,...
14622,9,tm1098060,NaN
14623,8,tm1098060,NaN
14624,12,NaN,ts271048


In [11]:
production_countries_exploded = titles[['id', 'production_countries']]

production_countries_exploded.loc[:, 'production_countries'] = production_countries_exploded['production_countries'].str.replace(r'\[|\]|\'', '', regex=True)
production_countries_exploded.loc[:, 'production_countries'] = production_countries_exploded['production_countries'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

production_countries_exploded = production_countries_exploded.explode('production_countries')

production_countries = production_countries_exploded[['production_countries']].drop_duplicates().reset_index(drop=True)
production_countries.index = production_countries.index + 1
production_countries['country_id'] = production_countries.index

production_countries_bridge = pd.merge(production_countries_exploded, production_countries, on='production_countries', how='left')
production_countries_bridge.rename(columns = {'index':'country_id', 'id':'title_id'}, inplace = True)
production_countries_bridge.rename_axis('index', inplace=True)
production_countries_bridge.index = production_countries_bridge.index + 1
production_countries_bridge.drop(columns=['production_countries'], inplace=True)

production_countries_bridge.loc[production_countries_bridge['title_id'].str.startswith('tm'), 'movie_id'] = production_countries_bridge['title_id']
production_countries_bridge.loc[production_countries_bridge['title_id'].str.startswith('ts'), 'show_id'] = production_countries_bridge['title_id']
production_countries_bridge.drop(columns=['title_id'], inplace=True)

production_countries.drop(columns=['country_id'], inplace=True)
production_countries.rename_axis('country_id', inplace=True)
production_countries.rename(columns = {'production_countries':'country'}, inplace = True)

del production_countries_exploded
production_countries

,country
country_id,
1,US
2,GB
3,JP
4,EG
5,DE
...,...
104,GT
105,MZ
106,AF


In [12]:
production_countries_bridge

,country_id,movie_id,show_id
index,,,
1,1,NaN,ts300399
2,1,tm84618,NaN
3,2,tm127384,NaN
4,2,tm70993,NaN
5,1,tm190788,NaN
...,...,...,...
6722,7,tm1108171,NaN
6723,46,tm1045018,NaN
6724,11,tm1045018,NaN


In [13]:
credits = pd.read_csv('../../datasets/raw_credits.csv')
credits

,index,person_id,id,name,character,role
0,0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...,...
77208,77208,1347054,tm1098060,Nnenna Rachael Okonkwo,Afinni,ACTOR
77209,77209,157590,tm1098060,Lucien Morgan,Dr. Ian Stones,ACTOR
77210,77210,129059,tm1098060,Magdalena Korpas,Jane,ACTOR
77211,77211,2050199,tm1098060,Mistura Olusanya,Nurse Titi,ACTOR


In [14]:
actors = credits[credits["role"] == "ACTOR"]

actors_bridge = actors[['id', 'person_id']].reset_index(drop=True)
actors_bridge.loc[actors_bridge['id'].str.startswith('tm'), 'movie_id'] = actors_bridge['id']
actors_bridge.loc[actors_bridge['id'].str.startswith('ts'), 'show_id'] = actors_bridge['id']
actors_bridge.drop(columns=['id'], inplace=True)
actors_bridge.rename(columns = {'person_id':'actor_id'}, inplace = True)
actors_bridge.rename_axis('index', inplace=True)
actors_bridge.index = actors_bridge.index + 1

actors = actors[['person_id', 'name']].drop_duplicates().reset_index(drop=True)
actors.index = actors.index + 1
actors.rename_axis('index', inplace=True)
actors.rename(columns = {'person_id':'actor_id', 'name':'actor'}, inplace = True)
actors

,actor_id,actor
index,,
1,3748,Robert De Niro
2,14658,Jodie Foster
3,7064,Albert Brooks
4,3739,Harvey Keitel
5,48933,Cybill Shepherd
...,...,...
50945,2050257,Pa Jimi Solanke
50946,1347054,Nnenna Rachael Okonkwo
50947,157590,Lucien Morgan


In [15]:
actors_bridge

,actor_id,movie_id,show_id
index,,,
1,3748,tm84618,NaN
2,14658,tm84618,NaN
3,7064,tm84618,NaN
4,3739,tm84618,NaN
5,48933,tm84618,NaN
...,...,...,...
72686,2050257,tm1098060,NaN
72687,1347054,tm1098060,NaN
72688,157590,tm1098060,NaN


In [16]:
directors = credits[credits['role'] == 'DIRECTOR']

directors_bridge = directors[['id', 'person_id']].reset_index(drop=True)
directors_bridge.loc[directors_bridge['id'].str.startswith('tm'), 'movie_id'] = directors_bridge['id']
directors_bridge.loc[directors_bridge['id'].str.startswith('ts'), 'show_id'] = directors_bridge['id']
directors_bridge.drop(columns=['id'], inplace=True)
directors_bridge.rename(columns = {'person_id':'director_id'}, inplace = True)
directors_bridge.rename_axis('index', inplace=True)
directors_bridge.index = directors_bridge.index + 1

directors = directors[['person_id', 'name']].drop_duplicates().reset_index(drop=True)
directors.index = directors.index + 1
directors.rename_axis('index', inplace=True)
directors.rename(columns = {'person_id':'director_id', 'name':'director'}, inplace = True)
directors

,director_id,director
index,,
1,3308,Martin Scorsese
2,11475,Terry Jones
3,11473,Terry Gilliam
4,14255,William Friedkin
5,56405,Don Siegel
...,...,...
3424,918476,Stanley D'Costa
3425,1790433,Easy Ferrer
3426,1278319,Kürşad Bayhan


In [17]:
directors_bridge

,director_id,movie_id,show_id
index,,,
1,3308,tm84618,NaN
2,11475,tm127384,NaN
3,11473,tm127384,NaN
4,11475,tm70993,NaN
5,14255,tm190788,NaN
...,...,...,...
4519,1790433,tm1040816,NaN
4520,1962840,tm1014599,NaN
4521,1278319,tm1108171,NaN


In [18]:
characters_exploded = credits[credits["role"] == "ACTOR"]
characters_exploded = characters_exploded[['id', 'person_id', 'character']]

characters_exploded.loc[:, 'character'] = characters_exploded['character'].str.replace(r'\[|\]|\'', '', regex=True)
characters_exploded.loc[:, 'character'] = characters_exploded['character'].str.split(' / ')

characters_exploded = characters_exploded.explode('character')

characters = characters_exploded[['character']].drop_duplicates().reset_index(drop=True)
characters.index = characters.index + 1
characters['character_id'] = characters.index

characters_bridge = pd.merge(characters_exploded, characters, on='character', how='left')
characters_bridge.rename(columns = {'id':'title_id', 'person_id':'actor_id'}, inplace = True)
characters_bridge.rename_axis('index', inplace=True)
characters_bridge.index = characters_bridge.index + 1
characters_bridge.drop(columns=['character'], inplace=True)

characters_bridge.loc[characters_bridge['title_id'].str.startswith('tm'), 'movie_id'] = characters_bridge['title_id']
characters_bridge.loc[characters_bridge['title_id'].str.startswith('ts'), 'show_id'] = characters_bridge['title_id']
characters_bridge.drop(columns=['title_id'], inplace=True)

characters.drop(columns=['character_id'], inplace=True)
characters.rename_axis('character_id', inplace=True)

del characters_exploded
characters

,character
character_id,
1,Travis Bickle
2,Iris Steensma
3,Tom
4,Matthew Sport Higgins
5,Betsy
...,...
47766,Akinola
47767,Akanjis Father
47768,Afinni


In [19]:
characters_bridge

,actor_id,character_id,movie_id,show_id
index,,,,
1,3748,1,tm84618,NaN
2,14658,2,tm84618,NaN
3,7064,3,tm84618,NaN
4,3739,4,tm84618,NaN
5,48933,5,tm84618,NaN
...,...,...,...,...
74295,2050257,47767,tm1098060,NaN
74296,1347054,47768,tm1098060,NaN
74297,157590,47769,tm1098060,NaN


In [20]:
imdb_info = titles[['id','imdb_id', 'imdb_score','imdb_votes']]
imdb_info.loc[imdb_info['id'].str.startswith('tm'), 'movie_id'] = imdb_info['id']
imdb_info.loc[imdb_info['id'].str.startswith('ts'), 'show_id'] = imdb_info['id']
imdb_info.rename_axis('index', inplace=True)
imdb_info.index = imdb_info.index + 1
imdb_info.drop(columns=['id'], inplace=True)
imdb_info

,imdb_id,imdb_score,imdb_votes,movie_id,show_id
index,,,,,
1,NaN,NaN,NaN,NaN,ts300399
2,tt0075314,8.3,795222.0,tm84618,NaN
3,tt0071853,8.2,530877.0,tm127384,NaN
4,tt0079470,8.0,392419.0,tm70993,NaN
5,tt0070047,8.1,391942.0,tm190788,NaN
...,...,...,...,...,...
5802,tt13857480,6.9,39.0,tm1014599,NaN
5803,NaN,NaN,NaN,tm1108171,NaN
5804,tt14620732,6.5,32.0,tm1045018,NaN


## Database Schema Definition
This part of the code defines SQLAlchemy model classes representing various tables in the database schema, including tables for movies, shows, genres, production countries, actors, directors, characters, and IMDB information. It establishes the structure and relationships between these tables and creates them in the database:

In [21]:
Base = declarative_base()

class Movies(Base):
    """
    Represents the movies table in the database.

    Attributes:
        index (int): Primary key for the movies table.
        movie_id (str): Unique identifier for the movie.
        title (str): Title of the movie.
        release_year (int): Year the movie was released.
        age_certification (str): Age certification for the movie.
        runtime (int): Runtime of the movie in minutes.
    """
    __tablename__ = "movies"

    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    movie_id = Column(String(128), index=True)
    title = Column(String(512))
    release_year = Column(Integer)
    age_certification = Column(String(512))
    runtime = Column(Integer)

    def __init__(self, movie_id, title, release_year, age_certification, runtime):
        """
        Initializes a Movies instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            title (str): Title of the movie.
            release_year (int): Year the movie was released.
            age_certification (str): Age certification for the movie.
            runtime (int): Runtime of the movie in minutes.
        """
        self.movie_id = movie_id
        self.title = title
        self.release_year = release_year 
        self.age_certification = age_certification
        self.runtime = runtime

    def __repr__(self):
        """
        Provides a string representation of the Movies instance.

        Returns:
            str: String representation of the Movies instance.
        """
        return (
        f"<Movies(movie_id={self.movie_id}, title={self.title}, release_year={self.release_year}, "
        f"age_certification={self.age_certification}, runtime={self.runtime})>"
        )

class Shows(Base):
    """
    Represents the shows table in the database.

    Attributes:
        index (int): Primary key for the shows table.
        show_id (str): Unique identifier for the show.
        title (str): Title of the show.
        release_year (int): Year the show was released.
        age_certification (str): Age certification for the show.
        runtime (int): Runtime of the show in minutes.
        seasons (int): Number of seasons of the show.
    """
    __tablename__ = "shows"

    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    show_id = Column(String(128), index=True)
    title = Column(String(512))
    release_year = Column(Integer)
    age_certification = Column(String(512))
    runtime = Column(Integer)
    seasons = Column(Integer)

    def __init__(self, show_id, title, release_year, age_certification, runtime, seasons):
        """
        Initializes a Shows instance.

        Args:
            show_id (str): Unique identifier for the show.
            title (str): Title of the show.
            release_year (int): Year the show was released.
            age_certification (str): Age certification for the show.
            runtime (int): Runtime of the show in minutes.
            seasons (int): Number of seasons of the show.
        """
        self.show_id = show_id
        self.title = title
        self.release_year = release_year 
        self.age_certification = age_certification
        self.runtime = runtime
        self.seasons = seasons

    def __repr__(self):
        """
        Provides a string representation of the Shows instance.

        Returns:
            str: String representation of the Shows instance.
        """
        return (
        f"<Shows(show_id={self.show_id}, title={self.title}, release_year={self.release_year}, "
        f"age_certification={self.age_certification}, runtime={self.runtime}, seasons={self.seasons})>"
        )

class Genres(Base):
    """
    Represents the genres table in the database.

    Attributes:
        genre_id (int): Primary key for the genres table.
        genre (str): Genre name.
    """
    __tablename__ = "genres"
    
    genre_id = Column(Integer, primary_key=True, unique=True, index=True)
    genre = Column(String(255))

    def __init__(self, genre_id, genre):
        """
        Initializes a Genres instance.

        Args:
            genre_id (int): Primary key for the genres table.
            genre (str): Genre name.
        """
        self.genre_id = genre_id
        self.genre = genre

    def __repr__(self):
        """
        Provides a string representation of the Genres instance.

        Returns:
            str: String representation of the Genres instance.
        """
        return f"<Genres(genre_id={self.genre_id}, genre={self.genre})>"

class GenresBridge(Base):
    """
    Represents the genres_bridge table in the database.

    Attributes:
        index (int): Primary key for the genres_bridge table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        genre_id (int): Foreign key for the genres table.
    """
    __tablename__ = "genres_bridge"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="GenresBridge.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="GenresBridge.show_id == Shows.show_id", uselist=False)
    genre = relationship("Genres", primaryjoin="GenresBridge.genre_id == Genres.genre_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_g_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_g_show_id'),
        ForeignKeyConstraint(['genre_id'], ['genres.genre_id'], name='fk_genre_id')
    )
    index = Column(Integer, primary_key=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    genre_id = Column(Integer, nullable=False)

    def __init__(self, movie_id, show_id, genre_id):
        """
        Initializes a GenresBridge instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            genre_id (int): Foreign key for the genres table.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.genre_id = genre_id

    def __repr__(self):
        """
        Provides a string representation of the GenresBridge instance.

        Returns:
            str: String representation of the GenresBridge instance.
        """
        return f"<GenresBridge(movie_id={self.movie_id}, show_id={self.show_id}, genre_id={self.genre_id})>"

class ProductionCountries(Base):
    """
    Represents the production_countries table in the database.

    Attributes:
        country_id (int): Primary key for the production_countries table.
        country (str): Name of the production country.
    """
    __tablename__ = "production_countries"
    
    country_id = Column(Integer, primary_key=True, unique=True, index=True)
    country = Column(String(255))

    def __init__(self, country_id, country):
        """
        Initializes a ProductionCountries instance.

        Args:
            country_id (int): Primary key for the production_countries table.
            country (str): Name of the production country.
        """
        self.country_id = country_id
        self.country = country

    def __repr__(self):
        """
        Provides a string representation of the ProductionCountries instance.

        Returns:
            str: String representation of the ProductionCountries instance.
        """
        return f"<Countries(country_id={self.country_id}, country={self.country})>"

class ProductionCountriesBridge(Base):
    """
    Represents the production_countries_bridge table in the database.

    Attributes:
        index (int): Primary key for the production_countries_bridge table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        country_id (int): Foreign key for the production_countries table.
    """
    __tablename__ = "production_countries_bridge"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="ProductionCountriesBridge.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="ProductionCountriesBridge.show_id == Shows.show_id", uselist=False)
    production_country = relationship("ProductionCountries", primaryjoin="ProductionCountriesBridge.country_id == ProductionCountries.country_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_pc_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_pc_show_id'),
        ForeignKeyConstraint(['country_id'], ['production_countries.country_id'], name='fk_country_id')
    )
    index = Column(Integer, primary_key=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    country_id = Column(Integer, nullable=False)

    def __init__(self, movie_id, show_id, country_id):
        """
        Initializes a ProductionCountriesBridge instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            country_id (int): Foreign key for the production_countries table.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.country_id = country_id

    def __repr__(self):
        """
        Provides a string representation of the ProductionCountriesBridge instance.

        Returns:
            str: String representation of the ProductionCountriesBridge instance.
        """
        return f"<ProductionCountriesBridge(movie_id={self.movie_id}, show_id={self.show_id}, country_id={self.country_id})>"

class Actors(Base):
    """
    Represents the actors table in the database.

    Attributes:
        index (int): Primary key for the actors table.
        actor_id (int): Unique identifier for the actor.
        actor (str): Name of the actor.
    """
    __tablename__ = "actors"

    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    actor_id = Column(Integer, index=True)
    actor = Column(String(512))

    def __init__(self, actor_id, actor):
        """
        Initializes an Actors instance.

        Args:
            actor_id (int): Unique identifier for the actor.
            actor (str): Name of the actor.
        """
        self.actor_id = actor_id
        self.actor = actor

    def __repr__(self):
        """
        Provides a string representation of the Actors instance.

        Returns:
            str: String representation of the Actors instance.
        """
        return f"<Actors(actor_id={self.actor_id}, actor={self.actor})>"
    
class ActorsBridge(Base):
    """
    Represents the actors_bridge table in the database.

    Attributes:
        index (int): Primary key for the actors_bridge table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        actor_id (int): Foreign key for the actors table.
    """
    __tablename__ = "actors_bridge"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="ActorsBridge.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="ActorsBridge.show_id == Shows.show_id", uselist=False)
    actor = relationship("ProductionCountries", primaryjoin="ActorsBridge.actor_id == Actors.actor_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_a_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_a_show_id'),
        ForeignKeyConstraint(['actor_id'], ['actors.actor_id'], name='fk_actor_id')
    )
    index = Column(Integer, primary_key=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    actor_id = Column(Integer, nullable=False)

    def __init__(self, movie_id, show_id, actor_id):
        """
        Initializes a ActorsBridge instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            actor_id (int): Foreign key for the actors table.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.actor_id = actor_id

    def __repr__(self):
        """
        Provides a string representation of the ActorsBridge instance.

        Returns:
            str: String representation of the ActorsBridge instance.
        """
        return f"<ActorsBridge(movie_id={self.movie_id}, show_id={self.show_id}, actor_id={self.actor_id})>"
    
class Directors(Base):
    """
    Represents the directors table in the database.

    Attributes:
        index (int): Primary key for the directors table.
        director_id (int): Unique identifier for the director.
        director (str): Name of the director.
    """
    __tablename__ = "directors"

    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    director_id = Column(Integer, index=True)
    director = Column(String(512))

    def __init__(self, director_id, director):
        """
        Initializes a Directors instance.

        Args:
            director_id (int): Unique identifier for the director.
            director (str): Name of the director.
        """
        self.director_id = director_id
        self.director = director

    def __repr__(self):
        """
        Provides a string representation of the Directors instance.

        Returns:
            str: String representation of the Directors instance.
        """
        return f"<Directors(director_id={self.director_id}, director={self.director})>"
    
class DirectorsBridge(Base):
    """
    Represents the directors_bridge table in the database.

    Attributes:
        index (int): Primary key for the directors_bridge table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        director_id (int): Foreign key for the actors table.
    """
    __tablename__ = "directors_bridge"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="DirectorsBridge.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="DirectorsBridge.show_id == Shows.show_id", uselist=False)
    director = relationship("Directors", primaryjoin="DirectorsBridge.director_id == Directors.director_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_d_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_d_show_id'),
        ForeignKeyConstraint(['director_id'], ['directors.director_id'], name='fk_director_id')
    )
    index = Column(Integer, primary_key=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    director_id = Column(Integer, nullable=False)

    def __init__(self, movie_id, show_id, director_id):
        """
        Initializes a DirectorsBridge instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            director_id (int): Foreign key for the actors table.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.director_id = director_id

    def __repr__(self):
        """
        Provides a string representation of the DirectorsBridge instance.

        Returns:
            str: String representation of the DirectorsBridge instance.
        """
        return f"<DirectorsBridge(movie_id={self.movie_id}, show_id={self.show_id}, director_id={self.director_id})>"
    
class Characters(Base):
    """
    Represents the characters table in the database.

    Attributes:
        character_id (int): Primary key for the characters table.
        character (str): Name of the character.
    """
    __tablename__ = "characters"
    
    character_id = Column(Integer, primary_key=True, unique=True, index=True)
    character = Column(String(255))

    def __init__(self, character_id, character):
        """
        Initializes a Characters instance.

        Args:
            character_id (int): Primary key for the characters table.
            character (str): Name of the character.
        """
        self.character_id = character_id
        self.character = character

    def __repr__(self):
        """
        Provides a string representation of the Characters instance.

        Returns:
            str: String representation of the Characters instance.
        """
        return f"<Characters(character_id={self.character_id}, character={self.character})>"

class CharactersBridge(Base):
    """
    Represents the characters_bridge table in the database.

    Attributes:
        index (int): Primary key for the characters_bridge table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        actor_id (int): Unique identifier for the actor.
        character_id (int): Foreign key for the characters table.
    """
    __tablename__ = "characters_bridge"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="CharactersBridge.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="CharactersBridge.show_id == Shows.show_id", uselist=False)
    actor = relationship('Actors', primaryjoin="CharactersBridge.actor_id == Actors.actor_id", uselist=False)
    character = relationship('Characters', primaryjoin="CharactersBridge.character_id == Characters.character_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_c_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_c_show_id'),
        ForeignKeyConstraint(['actor_id'], ['actors.actor_id'], name='fk_c_actor_id'),
        ForeignKeyConstraint(['character_id'], ['characters.character_id'], name='fk_character_id')
    )
    index = Column(Integer, primary_key=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    actor_id = Column(Integer)
    character_id = Column(Integer)

    def __init__(self, movie_id, show_id, actor_id, character_id):
        """
        Initializes a CharactersBridge instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            actor_id (int): Unique identifier for the actor.
            character_id (int): Foreign key for the characters table.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.actor_id = actor_id
        self.character_id = character_id

    def __repr__(self):
        """
        Provides a string representation of the CharactersBridge instance.

        Returns:
            str: String representation of the CharactersBridge instance.
        """
        return f"<CharactersBridge(movie_id={self.movie_id}, show_id={self.show_id}, actor_id={self.actor_id}, character_id={self.character_id})>"

class IMDBInfo(Base):
    """
    Represents the imdb_info table in the database.

    Attributes:
        index (int): Primary key for the imdb_info table.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        imdb_id (str): Unique identifier for the IMDB entry.
        imdb_score (float): IMDB score.
        imdb_votes (int): Number of votes on IMDB.
    """
    __tablename__ = "imdb_info"
    # Define conditional relationships
    movie = relationship("Movies", primaryjoin="IMDBInfo.movie_id == Movies.movie_id", uselist=False)
    show = relationship("Shows", primaryjoin="IMDBInfo.show_id == Shows.show_id", uselist=False)
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_imdb_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_imdb_show_id')
    )
    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    movie_id = Column(String(128))
    show_id = Column(String(128))
    imdb_id = Column(String(128))
    imdb_score = Column(Float)
    imdb_votes = Column(Integer)


    def __init__(self, movie_id, show_id, imdb_id, imdb_score, imdb_votes):
        """
        Initializes an IMDBInfo instance.

        Args:
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            imdb_id (str): Unique identifier for the IMDB entry.
            imdb_score (float): IMDB score.
            imdb_votes (int): Number of votes on IMDB.
        """
        self.movie_id = movie_id
        self.show_id = show_id
        self.imdb_id = imdb_id
        self.imdb_score = imdb_score
        self.imdb_votes = imdb_votes

    def __repr__(self):
        """
        Provides a string representation of the IMDBInfo instance.

        Returns:
            str: String representation of the IMDBInfo instance.
        """
        return f"<IMDBInfo(movie_id={self.movie_id}, show_id={self.show_id}, imdb_id={self.imdb_id}, imdb_score={self.imdb_score}, imdb_votes={self.imdb_votes})>"

class Predictions(Base):
    """
    Represents a prediction record in the database.

    Attributes:
        index (int): Unique identifier for the prediction record (auto-incrementing integer).
        timestamp (datetime): Datetime when the prediction was made (automatically set to UTC on creation).
        prediction_value (float): The actual predicted value.
    """

    __tablename__ = "predictions"

    index = Column(Integer, primary_key=True, unique=True, index=True, autoincrement=True)
    timestamp = Column(DateTime, default=lambda: datetime.now(timezone(timedelta(hours=2))))
    user_id = Column(String(64), nullable=False)
    prediction_value = Column(Float, nullable=False)

    def __init__(self, user_id, prediction_value):
        """
        Initializes a new prediction object.

        Args:
            timestamp (datetime): The datetime when the prediction was made.
            prediction_value (float): The predicted value.
        """

        self.timestamp =  datetime.now(timezone(timedelta(hours=2)))
        self.user_id =  user_id
        self.prediction_value = prediction_value

    def __repr__(self):
        """
        Returns a string representation of the prediction object.

        Returns:
            str: A string representation of the prediction object in the format
                `<Predictions(timestamp=..., prediction_value=...)>`.
        """

        return f"<Predictions(timestamp={self.timestamp}, user_id={self.user_id}, prediction_value={self.prediction_value})>"

class MoviesAndShowsView(Base_view):
    """
    Represents the movies_and_shows_view view in the database.

    Attributes:
        media_type (str): Type of media, either 'movie' or 'show'.
        index (int): Primary key for the view.
        movie_id (str): Unique identifier for the movie.
        show_id (str): Unique identifier for the show.
        title (str): Title of the media.
        release_year (int): Year of release.
        age_certification (str): Age certification.
        runtime (int): Runtime in minutes.
        seasons (int): Number of seasons (for shows).
        genre (str): Genre of the media.
        country (str): Country of production.
        director (str): Director of the media.
        actor (str): Actor in the media.
        character (str): Character played by the actor.
        imdb_score (float): IMDB score.
        imdb_votes (int): Number of votes on IMDB.
    """
    __tablename__ = "movies_and_shows_view"
    __table_args__ = {'autoload': False}
    media_type = Column(String, primary_key=True)
    index = Column(Integer, primary_key=True)
    movie_id = Column(String(128), nullable=True)
    show_id = Column(String(128), nullable=True)
    title = Column(String(256))
    release_year = Column(Integer)
    age_certification = Column(String(10))
    runtime = Column(Integer)
    seasons = Column(Integer, nullable=True)
    genre = Column(String(128))
    country = Column(String(128))
    director = Column(String(128))
    actor = Column(String(128))
    character = Column(String(128))
    imdb_score = Column(Float)
    imdb_votes = Column(Integer)
    
    __table_args__ = (
        ForeignKeyConstraint(['movie_id'], ['movies.movie_id'], name='fk_view_movie_id'),
        ForeignKeyConstraint(['show_id'], ['shows.show_id'], name='fk_view_show_id')
    )

    def __init__(self, media_type, index, movie_id, show_id, title, release_year, age_certification, runtime, seasons, genre, country, director, actor, character, imdb_score, imdb_votes):
        """
        Initializes a MoviesAndShowsView instance.

        Args:
            media_type (str): Type of media, either 'movie' or 'show'.
            index (int): Primary key for the view.
            movie_id (str): Unique identifier for the movie.
            show_id (str): Unique identifier for the show.
            title (str): Title of the media.
            release_year (int): Year of release.
            age_certification (str): Age certification.
            runtime (int): Runtime in minutes.
            seasons (int): Number of seasons (for shows).
            genre (str): Genre of the media.
            country (str): Country of production.
            director (str): Director of the media.
            actor (str): Actor in the media.
            character (str): Character played by the actor.
            imdb_score (float): IMDB score.
            imdb_votes (int): Number of votes on IMDB.
        """
        self.media_type = media_type
        self.index = index
        self.movie_id = movie_id
        self.show_id = show_id
        self.title = title
        self.release_year = release_year
        self.age_certification = age_certification
        self.runtime = runtime
        self.seasons = seasons
        self.genre = genre
        self.country = country
        self.director = director
        self.actor = actor
        self.character = character
        self.imdb_score = imdb_score
        self.imdb_votes = imdb_votes

    def __repr__(self):
        """
        Provides a string representation of the MoviesAndShowsView instance.

        Returns:
            str: String representation of the MoviesAndShowsView instance.
        """
        return f"<MoviesAndShowsView(media_type={self.media_type}, index={self.index}, movie_id={self.movie_id}, show_id={self.show_id}, title={self.title}, release_year={self.release_year}, age_certification={self.age_certification}, runtime={self.runtime}, seasons={self.seasons}, genre={self.genre}, country={self.country}, director={self.director}, actor={self.actor}, character={self.character}, imdb_score={self.imdb_score}, imdb_votes={self.imdb_votes})>"


Base.metadata.create_all(engine)

Also, let's create a view that allows us to access all of our table data in one place.

In [22]:
def create_view(engine):
    with engine.connect() as connection:
        sql = text("""
        CREATE VIEW movies_and_shows_view AS
        SELECT 
            'movie' AS media_type,
            m.index,
            m.movie_id,
            NULL AS show_id,
            m.title,
            m.release_year,
            m.age_certification,
            m.runtime,
            NULL AS seasons,
            g.genre,
            pc.country,
            d.director AS director,
            a.actor AS actor,
            c.character,
            i.imdb_score,
            i.imdb_votes
        FROM movies m
        LEFT JOIN genres_bridge gb ON m.movie_id = gb.movie_id
        LEFT JOIN genres g ON gb.genre_id = g.genre_id
        LEFT JOIN production_countries_bridge pcb ON m.movie_id = pcb.movie_id
        LEFT JOIN production_countries pc ON pcb.country_id = pc.country_id
        LEFT JOIN directors_bridge db ON m.movie_id = db.movie_id
        LEFT JOIN directors d ON db.director_id = d.director_id
        LEFT JOIN actors_bridge ab ON m.movie_id = ab.movie_id
        LEFT JOIN actors a ON ab.actor_id = a.actor_id
        LEFT JOIN characters_bridge cb ON m.movie_id = cb.movie_id AND ab.actor_id = cb.actor_id
        LEFT JOIN characters c ON cb.character_id = c.character_id
        LEFT JOIN imdb_info i ON m.movie_id = i.movie_id
        UNION ALL
        SELECT 
            'show' AS media_type,
            s.index,
            NULL AS movie_id,
            s.show_id,
            s.title,
            s.release_year,
            s.age_certification,
            s.runtime,
            s.seasons,
            g.genre,
            pc.country,
            d.director AS director,
            a.actor AS actor,
            c.character,
            i.imdb_score,
            i.imdb_votes
        FROM shows s
        LEFT JOIN genres_bridge gb ON s.show_id = gb.show_id
        LEFT JOIN genres g ON gb.genre_id = g.genre_id
        LEFT JOIN production_countries_bridge pcb ON s.show_id = pcb.show_id
        LEFT JOIN production_countries pc ON pcb.country_id = pc.country_id
        LEFT JOIN directors_bridge db ON s.show_id = db.show_id
        LEFT JOIN directors d ON db.director_id = d.director_id
        LEFT JOIN actors_bridge ab ON s.show_id = ab.show_id
        LEFT JOIN actors a ON ab.actor_id = a.actor_id
        LEFT JOIN characters_bridge cb ON s.show_id = cb.show_id AND ab.actor_id = cb.actor_id
        LEFT JOIN characters c ON cb.character_id = c.character_id
        LEFT JOIN imdb_info i ON s.show_id = i.show_id
        """)
        connection.execute(sql)

## Data Insertion

This code attempts to insert data into the corresponding tables in the database. If any SQLAlchemy error occurs during the insertion process, it logs the error to a file named 'data_error.log':

In [23]:
try:
    # Attempt to insert data into the database
    movies.to_sql('movies', engine, if_exists='append')
    shows.to_sql('shows', engine, if_exists='append')
    genres.to_sql('genres', engine, if_exists='append')
    genres_bridge.to_sql('genres_bridge', engine, if_exists='append')
    production_countries.to_sql('production_countries', engine, if_exists='append')
    production_countries_bridge.to_sql('production_countries_bridge', engine, if_exists='append')
    actors.to_sql('actors', engine, if_exists='append')
    actors_bridge.to_sql('actors_bridge', engine, if_exists='append')
    directors.to_sql('directors', engine, if_exists='append')
    directors_bridge.to_sql('directors_bridge', engine, if_exists='append')
    characters.to_sql('characters', engine, if_exists='append')
    characters_bridge.to_sql('characters_bridge', engine, if_exists='append')
    imdb_info.to_sql('imdb_info', engine, if_exists='append')
    logging.info("Data inserted successfully.")
    
    create_view(engine)
    logging.info("View 'movies_and_shows_view' created successfully.")
    
except exc.SQLAlchemyError as e:
    # Log the error
    logging.error("DataError: %s", e)

## Close the Connection

We free all resources by closing the connection. Remember that it is always important to close connections so that we can avoid unused connections taking up resources:

In [24]:
engine.dispose()